In [35]:
from IPython.display import display

import pandas as pd
import panel as pn
import param
import plotly.graph_objs as go

pn.extension('plotly')

In [5]:
df = pd.read_csv('data/test_pred.csv')
display(df.columns)

Index(['title', 'total_game_hours_last_two_weeks', 'num_groups', 'orig_url',
       'num_badges', 'review_url', 'num_found_funny', 'review', 'date_updated',
       'num_workshop_items', 'date_posted', 'found_helpful_percentage',
       'num_voted_helpfulness', 'achievement_progress', 'profile_url',
       'num_found_helpful', 'steam_id_number', 'friend_player_level',
       'num_found_unhelpful', 'total_game_hours', 'username', 'num_guides',
       'rating', 'num_friends', 'num_screenshots', 'num_comments',
       'num_reviews', 'num_games_owned', 'pred_rfr_untuned',
       'pred_xgbr_untuned', 'pred_krr_untuned', 'pred_linr_untuned',
       'pred_lstm_untuned', 'pred_lstm2_untuned'],
      dtype='object')

In [14]:
regr_algs = [x for x in df.columns if x.startswith('pred')]
regr_alg_names = ['Random Forest Regressor', 'XGBoost Regressor', 'Kernel Ridge Regressor',
                  'Linear Regressor', 'LSTM regressor 1', 'LSTM regressor 2']
regr_algs_names = dict(zip(regr_algs, regr_alg_names))
regr_algs_names

{'pred_rfr_untuned': 'Random Forest Regressor',
 'pred_xgbr_untuned': 'XGBoost Regressor',
 'pred_krr_untuned': 'Kernel Ridge Regressor',
 'pred_linr_untuned': 'Linear Regressor',
 'pred_lstm_untuned': 'LSTM regressor 1',
 'pred_lstm2_untuned': 'LSTM regressor 2'}

In [41]:
class UsefulnessRegression(param.Parameterized):
    def get_regressor_map():
        regr_algs = [x for x in df.columns if x.startswith('pred')]
        regr_alg_names = ['Random Forest Regressor', 'XGBoost Regressor', 'Kernel Ridge Regressor',
                        'Linear Regressor', 'LSTM regressor 1', 'LSTM regressor 2']
        regr_algs_names = dict(zip(regr_alg_names, regr_algs))
        return regr_algs_names
    regr_names_algs = get_regressor_map()
    regr_names = list(regr_names_algs.keys())
    game_names = sorted(list(df.title.unique()))
    regressor = param.ObjectSelector(default=regr_names[0], objects=regr_names)
    game_titles = param.ObjectSelector(default='All', objects=['All']+game_names)

    def get_selected_data(self):
        df_sel = df[['found_helpful_percentage', self.regr_names_algs[self.regressor]]].copy()
        if self.game_titles != 'All':
            df_sel = df_sel[df_sel.title == self.game_titles].copy()
        return df_sel
    
    def plotly_view(self):
        data = self.get_selected_data()
        title = 'Predicted vs. Truth Fraction of Usefulness Votes'

        # traces = []
        # for continent, df in data.groupby('continent'):
        #     marker=dict(symbol='circle', sizemode='area', sizeref=0.1, size=df['size'], line=dict(width=2))
        #     traces.append(go.Scatter(x=df.gdpPercap, y=df.lifeExp, mode='markers', marker=marker, name=continent))
        
        # axis_opts = dict(gridcolor='rgb(255, 255, 255)', zerolinewidth=1, ticklen=5, gridwidth=2)
        # layout = go.Layout(title=title, showlegend=self.show_legend, width=550,
        #     xaxis=dict(title=self.xlabel, type='log', **axis_opts),
        #     yaxis=dict( title=self.ylabel, **axis_opts))
        
        # return go.Figure(data=traces, layout=layout)
        # fig_scat = go.FigureWidget()
        # fig_scat.add_scatter(
        #     x=data.found_helpful_percentage,
        #     y=data[self.regr_names_algs[self.regressor]],
        #     mode='markers'
        # )
        traces = []
        traces.append(go.Scatter(x=data.found_helpful_percentage, y=data[self.regr_names_algs[self.regressor]], mode='markers'))
        layout = go.Layout(title=title)
        return go.Figure(data=traces, layout=layout)
my_regr = UsefulnessRegression()

In [43]:
# It works!
pn.Row(my_regr.param, my_regr.plotly_view)

BokehModel(combine_events=True, render_bundle={'docs_json': {'645303fe-a398-4961-86d5-aabba66c6cf1': {'defs': …

Row
    [0] Column(margin=5, name='UsefulnessRegression', width=300)
        [0] StaticText(value='<b>UsefulnessRegression</...)
        [1] Select(name='Regressor', options=OrderedDict([('Random Fore...]), value='Random Forest Regressor')
        [2] Select(name='Game titles', options=OrderedDict([('All', ...]), value='All')
    [1] ParamMethod(method)